In [521]:
import pandas as pd
import numpy as np

df = pd.read_csv("seoul_crime2.csv", encoding="utf-8", delimiter="\t", header=1, thousands=",")
df.head()

,기간,자치구,발생,검거,발생.1,검거.1,발생.2,검거.2,발생.3,검거.3,발생.4,검거.4,발생.5,검거.5
0,2014,합계,130674,79061,158,152,343,299,5462,4957,59393,19120,65318,54533
1,2014,종로구,5021,4610,3,7,12,15,226,948,2272,1281,2508,2359
2,2014,중구,5231,3188,6,5,13,13,221,161,2576,887,2415,2122
3,2014,용산구,3799,2340,1,2,7,7,213,169,1560,522,2018,1640
4,2014,성동구,3582,2048,1,1,5,6,141,103,1753,529,1682,1409


In [522]:
df.columns = ['기간', '자치구', '발생합계', '검거합계', '살인발생', '살인검거', '강도발생', '강도검거', '강간발생', '강간검거',\
              '절도발생', '절도검거', '폭력발생', '폭력검거']
df.columns

Index(['기간', '자치구', '발생합계', '검거합계', '살인발생', '살인검거', '강도발생', '강도검거', '강간발생',
       '강간검거', '절도발생', '절도검거', '폭력발생', '폭력검거'],
      dtype='object')

In [523]:
df = df[df["자치구"] != "합계"]

df.loc[80, "살인검거"] = 0
df["살인검거"] = df["살인검거"].astype("int64")


In [524]:
df

,기간,자치구,발생합계,검거합계,살인발생,살인검거,강도발생,강도검거,강간발생,강간검거,절도발생,절도검거,폭력발생,폭력검거
1,2014,종로구,5021,4610,3,7,12,15,226,948,2272,1281,2508,2359
2,2014,중구,5231,3188,6,5,13,13,221,161,2576,887,2415,2122
3,2014,용산구,3799,2340,1,2,7,7,213,169,1560,522,2018,1640
4,2014,성동구,3582,2048,1,1,5,6,141,103,1753,529,1682,1409
5,2014,광진구,6268,3531,8,9,16,22,249,199,3167,1001,2828,2300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2019,관악구,5328,3810,13,10,10,11,408,335,2223,1085,2674,2369
152,2019,서초구,5542,3750,5,5,7,5,616,412,2270,1118,2644,2210
153,2019,강남구,7304,5069,5,3,5,6,666,562,2970,1339,3658,3159
154,2019,송파구,5698,3799,7,8,10,10,273,230,2416,1016,2992,2535


In [525]:
df["발생확인"] = df["살인발생"] + df["강도발생"] + df["강간발생"] + df["절도발생"] + df["폭력발생"] 
df["검거확인"] = df["살인검거"] + df["강도검거"] + df["강간검거"] + df["절도검거"] + df["폭력검거"] 
df[["발생합계", "발생확인", "검거합계", "검거확인"]].head()

,발생합계,발생확인,검거합계,검거확인
1,5021,5021,4610,4610
2,5231,5231,3188,3188
3,3799,3799,2340,2340
4,3582,3582,2048,2048
5,6268,6268,3531,3531


In [526]:
df[df["자치구"]=="강남구"].sum()

기간                   12099
자치구     강남구강남구강남구강남구강남구강남구
발생합계                 48154
검거합계                 32485
살인발생                    53
살인검거                    47
강도발생                   136
강도검거                   129
강간발생                  3350
강간검거                  2746
절도발생                 20436
절도검거                  8800
폭력발생                 24179
폭력검거                 20763
발생확인                 48154
검거확인                 32485
dtype: object

In [527]:
for i in df[df["자치구"]=="강남구"].iloc[:len(df), 2:3].sum():
    print(i)

48154


In [528]:
gu = []
for i in df["자치구"]:
    value = str(df[df["자치구"]==i].iloc[:len(df), 2:3].sum())
    gu.append({"자치구":i, "범죄발생합계":value})

gu = list(map(dict, set(tuple(i.items()) for i in gu)))

In [529]:
df_total = pd.DataFrame(gu)
df_total["범죄발생합계"] = df_total["범죄발생합계"].str.replace("발생합계", "")
df_total["범죄발생합계"] = df_total["범죄발생합계"].str.replace("\ndtype: int64", "").astype("int64")

coordinate = pd.read_csv("seoul_coordinate.csv")
df_cd = coordinate.drop("시군구코드", axis=1)
df_cd.rename(columns={"시군구명" : "자치구"}, inplace=True)

In [531]:
df_total_cd = pd.merge(df_total, df_cd, how="left", on="자치구")
df_total_cd.to_csv("crime_total_coordinate.csv", index=False)
df_total_cd

,자치구,범죄발생합계,위도,경도
0,강동구,27292,37.549208,127.146482
1,양천구,23893,37.527062,126.856153
2,금천구,20931,37.460097,126.900155
3,동대문구,25217,37.583801,127.050700
4,은평구,26030,37.617612,126.922700
5,영등포구,37821,37.520641,126.913924
6,용산구,22540,37.531101,126.981074
7,도봉구,14584,37.665861,127.031767
8,광진구,30071,37.548144,127.085753
9,노원구,27354,37.655264,127.077120
